<a href="https://colab.research.google.com/github/hy0park/ML-Projects/blob/main/Loan_Class_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_decision_forests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 30.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import tensorflow_decision_forests as tfdf

In [ ]:
import tensorflow as tf

import cProfile

In [ ]:
tf.executing_eagerly()

True

#Example Code

In [ ]:
path = "https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins.csv"
pandas_dataset = pd.read_csv(path)

# Display the first 3 examples.
pandas_dataset.head(3)

label = "species"

classes = list(pandas_dataset[label].unique())
print(f"Label classes: {classes}")
# >> Label classes: ['Adelie', 'Gentoo', 'Chinstrap']

pandas_dataset[label] = pandas_dataset[label].map(classes.index)

pandas_dataset.fillna(0)

np.random.seed(1)
# Use the ~10% of the examples as the testing set
# and the remaining ~90% of the examples as the training set.
test_indices = np.random.rand(len(pandas_dataset)) < 0.1
pandas_train_dataset = pandas_dataset[~test_indices]
pandas_test_dataset = pandas_dataset[test_indices]

print("Training examples: ", len(pandas_train_dataset))
# >> Training examples: 309

print("Testing examples: ", len(pandas_test_dataset))
# >> Testing examples: 35

tf_train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(pandas_train_dataset, label=label, batch_size=100)
model = tfdf.keras.CartModel()
model.fit(tf_train_dataset)

tfdf.model_plotter.plot_model_in_colab(model, max_depth=10)

Label classes: ['Adelie', 'Gentoo', 'Chinstrap']
Training examples:  309
Testing examples:  35
Use /tmp/tmp3vghev98 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:08.505345. Found 309 examples.
Training model...
Model trained in 0:00:00.040633
Compiling model...
Model compiled.


In [ ]:
model.compile("accuracy")
print("Train evaluation: ", model.evaluate(tf_train_dataset, return_dict=True))
# >> Train evaluation:  {'loss': 0.0, 'accuracy': 0.96116}

tf_test_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(pandas_test_dataset, label=label)
print("Test evaluation: ", model.evaluate(tf_test_dataset, return_dict=True))
# >> Test evaluation:  {'loss': 0.0, 'accuracy': 0.97142}

4/4 [==============================] - 1s 9ms/step - loss: 0.0000e+00 - accuracy: 0.9709
Train evaluation:  {'loss': 0.0, 'accuracy': 0.9708737730979919}
1/1 [==============================] - 0s 118ms/step - loss: 0.0000e+00 - accuracy: 0.9714
Test evaluation:  {'loss': 0.0, 'accuracy': 0.9714285731315613}


#Classify Loan Classes

##Train Data Cleaning

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('train').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
df = pd.DataFrame.from_records(rows)

# Display the first 3 examples.
df.head(3)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
1,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.9,15,부채 통합,0,0,0,0,0,C
2,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060,0,0,B


In [ ]:
df = df.loc[1:]

df.rename(columns={
    0: 'ID',
    1: 'Amount Of Loan', #대출금액
    2: 'Loan Period', #대출기간
    3: 'Working Period', #근로기간
    4: 'Home Own', #주택소유상태
    5: 'Yearly Earning', #연간소득
    6: 'Rate of Earning to Loan', #부채_대비_소득_비율
    7: 'Total Acnt Num', #총계좌수
    8: 'Loan Purpose', #대출목적
    9: 'Delay Num', #최근_2년간_연체_횟수
    10: 'Total Repay', #총상환원금
    11: 'Total Interest', #총상환이자
    12: 'Total Delay', #총연체금액
    13: 'Delay Acnt Num', #연체계좌수
    14: 'Loan Class', #대출등급
}, inplace=True)
df.head(3)

<ipython-input-9-92d3e0b8a83c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={


,ID,Amount Of Loan,Loan Period,Working Period,Home Own,Yearly Earning,Rate of Earning to Loan,Total Acnt Num,Loan Purpose,Delay Num,Total Repay,Total Interest,Total Delay,Delay Acnt Num,Loan Class
1,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.9,15,부채 통합,0,0,0,0,0,C
2,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060,0,0,B
3,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.6,14,부채 통합,0,928644,151944,0,0,A


In [ ]:
# Data Cleaning of column Working Period
df['Working Period'].replace(['<1 year', '< 1 year', '1 year', '1 years', '2 years','3 years', '4 years', '5 years'
, '6 years', '7 years', '8 years', '9 years', '10+years', '10+ years'], [0,0,1,1,2,3,4,5,6,7,8,9,10,10], inplace=True)
df.head(3)

,ID,Amount Of Loan,Loan Period,Working Period,Home Own,Yearly Earning,Rate of Earning to Loan,Total Acnt Num,Loan Purpose,Delay Num,Total Repay,Total Interest,Total Delay,Delay Acnt Num,Loan Class
1,TRAIN_00000,12480000,36 months,6,RENT,72000000,18.9,15,부채 통합,0,0,0,0,0,C
2,TRAIN_00001,14400000,60 months,10,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060,0,0,B
3,TRAIN_00002,12000000,36 months,5,MORTGAGE,96000000,8.6,14,부채 통합,0,928644,151944,0,0,A


In [ ]:
# New column using Working Period: less than 4 | less than 10 | over 10 | Unknown

df['Working Group'] = df['Working Period'].copy()
df['Working Group'].replace([0, 1, 2, 3], ['less than 4','less than 4', 'less than 4','less than 4'], inplace=True)
df['Working Group'].replace([4, 5, 6, 7, 8, 9], ['less than 10','less than 10', 'less than 10','less than 10', 'less than 10','less than 10'], inplace=True)
df['Working Group'].replace([10], ['over 10 years'], inplace=True)

In [ ]:
# Data Cleaning of column Home Own. There is just 1 data having 'ANY' value in Home Own column.

df = df.loc[df['Home Own'] != 'ANY']
df.head(3)

,ID,Amount Of Loan,Loan Period,Working Period,Home Own,Yearly Earning,Rate of Earning to Loan,Total Acnt Num,Loan Purpose,Delay Num,Total Repay,Total Interest,Total Delay,Delay Acnt Num,Loan Class,Working Group
1,TRAIN_00000,12480000,36 months,6,RENT,72000000,18.9,15,부채 통합,0,0,0,0,0,C,less than 10
2,TRAIN_00001,14400000,60 months,10,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060,0,0,B,over 10 years
3,TRAIN_00002,12000000,36 months,5,MORTGAGE,96000000,8.6,14,부채 통합,0,928644,151944,0,0,A,less than 10


In [ ]:
df['Amount Of Loan'] = df['Amount Of Loan'].astype(int)
df['Total Acnt Num'] = df['Total Acnt Num'].astype(int)
df['Delay Num'] = df['Delay Num'].astype(int)
df['Total Repay'] = df['Total Repay'].astype(int)
df['Total Delay'] = df['Total Delay'].astype(int)
df['Delay Acnt Num'] = df['Delay Acnt Num'].astype(int)

In [ ]:
#data clipping with column : total acnt num, earning, rate of earning

df['Yearly Earning'] = df['Yearly Earning'].astype(float)
df['Rate of Earning to Loan'] = df['Rate of Earning to Loan'].astype(float)
df['Total Acnt Num'] = df['Total Acnt Num'].astype(int)

df['Yearly Earning'] = df['Yearly Earning'].clip(lower=0, upper=500000000)
df['Rate of Earning to Loan'] = df['Rate of Earning to Loan'].clip(lower=0, upper=200)
df['Total Acnt Num'] = df['Total Acnt Num'].clip(lower=0, upper=50)

df.head(3)

,ID,Amount Of Loan,Loan Period,Working Period,Home Own,Yearly Earning,Rate of Earning to Loan,Total Acnt Num,Loan Purpose,Delay Num,Total Repay,Total Interest,Total Delay,Delay Acnt Num,Loan Class,Working Group
1,TRAIN_00000,12480000,36 months,6,RENT,72000000.0,18.90,15,부채 통합,0,0,0,0,0,C,less than 10
2,TRAIN_00001,14400000,60 months,10,MORTGAGE,130800000.0,22.33,21,주택 개선,0,373572,234060,0,0,B,over 10 years
3,TRAIN_00002,12000000,36 months,5,MORTGAGE,96000000.0,8.60,14,부채 통합,0,928644,151944,0,0,A,less than 10


##Test Data Cleaning

In [ ]:
worksheet = gc.open('test').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
test = pd.DataFrame.from_records(rows)

# Display the first 3 examples.
test.head(3)

In [ ]:
test = test.loc[1:]

test.rename(columns={
    0: 'ID',
    1: 'Amount Of Loan', #대출금액
    2: 'Loan Period', #대출기간
    3: 'Working Period', #근로기간
    4: 'Home Own', #주택소유상태
    5: 'Yearly Earning', #연간소득
    6: 'Rate of Earning to Loan', #부채_대비_소득_비율
    7: 'Total Acnt Num', #총계좌수
    8: 'Loan Purpose', #대출목적
    9: 'Delay Num', #최근_2년간_연체_횟수
    10: 'Total Repay', #총상환원금
    11: 'Total Interest', #총상환이자
    12: 'Total Delay', #총연체금액
    13: 'Delay Acnt Num', #연체계좌수
    14: 'Loan Class', #대출등급
}, inplace=True)
test.head(3)

<ipython-input-18-08558aec2ea0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.rename(columns={


,ID,Amount Of Loan,Loan Period,Working Period,Home Own,Yearly Earning,Rate of Earning to Loan,Total Acnt Num,Loan Purpose,Delay Num,Total Repay,Total Interest,Total Delay,Delay Acnt Num
1,TEST_00000,16800000,36 months,8 years,MORTGAGE,132000000,19.64,12,주택 개선,0,394692,146604,0,0
2,TEST_00001,8400000,36 months,5 years,RENT,89971200,15.84,25,부채 통합,0,0,0,0,0
3,TEST_00002,17280000,36 months,6 years,RENT,150000000,8.41,20,신용 카드,0,1786980,281820,0,0


In [ ]:
# Data Cleaning of column Working Period
test['Working Period'].replace(['<1 year', '< 1 year', '1 year', '1 years', '2 years','3 years', '4 years', '5 years'
, '6 years', '7 years', '8 years', '9 years', '10+years', '10+ years'], [0,0,1,1,2,3,4,5,6,7,8,9,10,10], inplace=True)
test.head(3)

<ipython-input-19-d56513c03f52>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Working Period'].replace(['<1 year', '< 1 year', '1 year', '1 years', '2 years','3 years', '4 years', '5 years'


,ID,Amount Of Loan,Loan Period,Working Period,Home Own,Yearly Earning,Rate of Earning to Loan,Total Acnt Num,Loan Purpose,Delay Num,Total Repay,Total Interest,Total Delay,Delay Acnt Num
1,TEST_00000,16800000,36 months,8,MORTGAGE,132000000,19.64,12,주택 개선,0,394692,146604,0,0
2,TEST_00001,8400000,36 months,5,RENT,89971200,15.84,25,부채 통합,0,0,0,0,0
3,TEST_00002,17280000,36 months,6,RENT,150000000,8.41,20,신용 카드,0,1786980,281820,0,0


In [ ]:
# New column using Working Period: less than 4 | less than 10 | over 10 | Unknown

test['Working Group'] = test['Working Period'].copy()
test['Working Group'].replace([0, 1, 2, 3], ['less than 4','less than 4', 'less than 4','less than 4'], inplace=True)
test['Working Group'].replace([4, 5, 6, 7, 8, 9], ['less than 10','less than 10', 'less than 10','less than 10', 'less than 10','less than 10'], inplace=True)
test['Working Group'].replace([10], ['over 10 years'], inplace=True)

<ipython-input-20-788f6b8f82cb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Working Group'] = test['Working Period'].copy()
<ipython-input-20-788f6b8f82cb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Working Group'].replace([0, 1, 2, 3], ['less than 4','less than 4', 'less than 4','less than 4'], inplace=True)
<ipython-input-20-788f6b8f82cb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

In [ ]:
test['Amount Of Loan'] = test['Amount Of Loan'].astype(int)
test['Total Acnt Num'] = test['Total Acnt Num'].astype(int)
test['Delay Num'] = test['Delay Num'].astype(int)
test['Total Repay'] = test['Total Repay'].astype(int)
test['Total Delay'] = test['Total Delay'].astype(int)
test['Delay Acnt Num'] = test['Delay Acnt Num'].astype(int)
test['Yearly Earning'] = test['Yearly Earning'].astype(float)
test['Rate of Earning to Loan'] = test['Rate of Earning to Loan'].astype(float)
test['Total Acnt Num'] = test['Total Acnt Num'].astype(int)

<ipython-input-21-5a8ee19c7f61>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Amount Of Loan'] = test['Amount Of Loan'].astype(int)
<ipython-input-21-5a8ee19c7f61>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Total Acnt Num'] = test['Total Acnt Num'].astype(int)
<ipython-input-21-5a8ee19c7f61>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

#Decision Tree

In [ ]:
label = "Loan Class"

classes = list(df[label].unique())
print(f"Label classes: {classes}")
# >> Label classes: ['A', 'B', 'C', 'D', 'F', 'G', 'E']

df[label] = df[label].map(classes.index)

Label classes: ['C', 'B', 'A', 'D', 'F', 'G', 'E']


In [ ]:
np.random.seed(1)
# Use the ~10% of the examples as the testing set
# and the remaining ~90% of the examples as the training set.
test_indices = np.random.rand(len(df)) < 0.1 #Validation set
pandas_train_dataset = df[~test_indices]
pandas_test_dataset = df[test_indices]

print("Training examples: ", len(pandas_train_dataset))
# >> Training examples: 86525

print("Testing examples: ", len(pandas_test_dataset))
# >> Testing examples: 9768

Training examples:  86525
Testing examples:  9768


In [ ]:
pandas_train_dataset = pandas_train_dataset.drop('Working Period', axis=1)
pandas_test_dataset = pandas_test_dataset.drop('Working Period', axis=1)

In [ ]:
# pandas_train_dataset = pandas_train_dataset.rename(columns={'Loan Class': 'label'})
pandas_test_dataset = pandas_test_dataset.rename(columns={'Loan Class': 'label'})

In [ ]:
tf_train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(pandas_train_dataset, label='label')
model = tfdf.keras.CartModel(split_axis="SPARSE_OBLIQUE")
model.fit(tf_train_dataset)

Use /tmp/tmp5u_fwhuy as temporary training directory
Reading training dataset...
Training dataset read in 0:00:01.136203. Found 86525 examples.
Training model...
Model trained in 0:00:15.997310
Compiling model...
Model compiled.


In [ ]:
tfdf.model_plotter.plot_model_in_colab(model, max_depth=3)

In [ ]:
model.compile("accuracy")
print("Train evaluation: ", model.evaluate(tf_train_dataset, return_dict=True))
# >> Train evaluation:  {'loss': 0.0, 'accuracy': 0.5177}

tf_test_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(pandas_test_dataset, label='label')
print("Test evaluation: ", model.evaluate(tf_test_dataset, return_dict=True))
# >> Test evaluation:  {'loss': 0.0, 'accuracy': 0.4710}

87/87 [==============================] - 1s 12ms/step - loss: 0.0000e+00 - accuracy: 0.5177
Train evaluation:  {'loss': 0.0, 'accuracy': 0.517688512802124}


10/10 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - accuracy: 0.4710
Test evaluation:  {'loss': 0.0, 'accuracy': 0.47102785110473633}


In [ ]:
# Create another model with specified hyper-parameters
model = tfdf.keras.GradientBoostedTreesModel(
    num_trees=500,
    growing_strategy="BEST_FIRST_GLOBAL",
    max_depth=8,
    split_axis="SPARSE_OBLIQUE",
    )

model.fit(tf_train_dataset)

# Evaluate the model
model.compile(metrics=["accuracy"])
print("Train evaluation: ", model.evaluate(tf_train_dataset, return_dict=True))
# >> Train evaluation:  {'loss': 0.0, 'accuracy': 0.5177}

tf_test_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(pandas_test_dataset, label='label')
print("Test evaluation: ", model.evaluate(tf_test_dataset, return_dict=True))
# >> Test evaluation:  {'loss': 0.0, 'accuracy': 0.4710}


print(model.evaluate(tf_test_dataset))
# >> 0.7555282

Use /tmp/tmp4w3wqhm8 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:01.940754. Found 86525 examples.
Training model...
Model trained in 1:34:24.183568
Compiling model...
Model compiled.
10/10 [==============================] - 3s 321ms/step - loss: 0.0000e+00 - accuracy: 0.7555
[0.0, 0.7555282711982727]
